## Setup thư viện

In [36]:
!pip install bitsandbytes
!pip install datasets
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install loralib
!pip install einops
!pip install googletrans==3.1.0a0

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-gen7jb4j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gen7jb4j
  Resolved https://github.com/huggingface/transformers.git to commit 5fabebdb7d4f9ee5a6459f7c0dcde0b1901f6205
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-xkv3v2ly
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-xkv3v2ly
  Resolved https://github.com/huggingface/peft.git to commit 56773b9a92b141111d65fe3548d0c30233358868
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-_p_pvjqz
  Runni

## Import thư viện

In [37]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers

from googletrans import Translator
from pprint import pprint
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


## Khởi tạo model

In [38]:
MODEL_NAME = "vilm/vinallama-7b-chat"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "up_proj",
        "o_proj",
        "k_proj",
        "down_proj",
        "gate_proj",
        "v_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Điều chỉnh một vài tham số

In [39]:
generation_config = model.generation_config
generation_config.max_length = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id


Tạo prompt

In [40]:
prompt = """<|im_start|>system
Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời
<|im_end|>
<|im_start|>user
{your task}
<|im_end|>
<|im_start|>assistant
 """.strip()


## Chạy thử một câu hỏi về trường, tất nhiên model sẽ không biết :V

In [41]:
prompt = """<|im_start|>system Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời <|im_end|> <|im_start|>user Trường đại học Khoa học Tự nhiên DHQG-HCM có bao nhiêu cơ sở <|im_end|> <|im_start|>assistant
 """.strip()


In [42]:
%%time
device = 'cuda' if torch.cuda.is_available() else 'cpu'

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<|im_start|> system Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời   <|im_start|> user Trường đại học Khoa học Tự nhiên DHQG-HCM có bao nhiêu cơ sở   <|im_start|> assistant
Không có đủ thông tin để cung cấp số lượng cơ sở mà Trường Đại học Khoa học Tự nhiên DHQG-HCM có. Vui lòng cung cấp thêm thông tin hoặc làm rõ câu hỏi.
CPU times: user 6.73 s, sys: 233 ms, total: 6.96 s
Wall time: 7.06 s


## Finetune cũng giống như trainning vậy, một pre-trained model có thể tốt với các thông tin phổ thông nhưng khi hỏi về chủ đề hay tác vụ cụ thể nào đấy thì nó sẽ không tốt. Vì vậy mình sẽ tiến hành finetune lại model trên một tập data trường ( ở đâu ra tập data đó thì không biết)

### Xử dụng một tập data siêu nhỏ :V , crawl từ Q&A fit để trainning tạm vì không có nguồn nào tốt hơn.

In [43]:
import json
with open("Q&A_fit_data.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
data = []
temp = {}
for line in lines:
    line = line.strip()
    if line.startswith("Title:"):
        if temp:
            data.append(temp)
        temp = {"prompt": line[7:].strip()}
    elif line.startswith("Question:"):
        temp["prompt"] += " " + line[10:].strip()
    elif line.startswith("Answer:"):
        temp["response"] = line[8:].strip()
with open("Q&A_dataset.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)


In [58]:
from datasets import load_dataset

data = load_dataset('json', data_files='Q&A_dataset.json')


Generating train split: 0 examples [00:00, ? examples/s]

### Không có chia train với test gì ở đây đâu, data đã ít mà còn không liên quan với nhau nên ta sẽ dùng hết vô để train :V

In [59]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt'],
        num_rows: 724
    })
})


## Bây giờ mình sẽ tiếp tục tạo lại hàm prompt, vì nó cần đúng với format của data.

In [60]:
def generate_prompt(data_point):
  vn_prompt = data_point['prompt']
  vn_response = data_point['response']
  return f"""
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
{vn_prompt}
<|im_end|>
<|im_start|>assistant
{vn_response}
""".strip()

## Xây dựng hàm tokenizer

In [61]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

#hàm tạo câu prompt với điểm dữ liệu gồmcặp(response,prompt)đầu vào, rồi tokenize cái prompt

In [62]:
# Tokenize bộ data

data = data['train'].shard(num_shards=50, index=0).filter(lambda sample: sample['response'] != '' and sample['prompt'] != '').shuffle().map(generate_and_tokenize_prompt)


Filter:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

## Bây giờ tới công đoạn finetune, nó khá giống như lúc ae mình cấu hình mấy cái layer, hàm optimizer, ... trong mạng CNN vậy.

In [63]:
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir="experiments",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,4.024000
2,3.758000
3,3.674600


TrainOutput(global_step=3, training_loss=3.8188732465108237, metrics={'train_runtime': 22.1158, 'train_samples_per_second': 0.678, 'train_steps_per_second': 0.136, 'total_flos': 81919443664896.0, 'train_loss': 3.8188732465108237, 'epoch': 0.8})

In [65]:
%%time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Hỏi một câu ná ná trong tập dataset để xem nó có trả lời ổn không.
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Dạ cho em hỏi về 2 chuyên ngành An toàn thông tin và Công nghệ tri thức khác nhau giữa 2 chuyên ngành này có được không ạ
<|im_end|>
<|im_start|>assistant
""".strip()


encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


<|im_start|> system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
 
<|im_start|> user
Dạ cho em hỏi về 2 chuyên ngành An toàn thông tin và Công nghệ tri thức khác nhau giữa 2 chuyên ngành này có được không ạ
 
<|im_start|> assistant

CPU times: user 1.2 s, sys: 531 ms, total: 1.73 s
Wall time: 1.73 s


## Nó không phản hồi được :V , LMAO. Thử test lại với một tập data lớn hơn chuẩn hơn.

In [66]:
data = load_dataset('alespalla/chatbot_instruction_prompts')

In [67]:
print(data)

DatasetDict({
    train: Dataset({
        features: ['response', 'prompt'],
        num_rows: 258042
    })
    test: Dataset({
        features: ['response', 'prompt'],
        num_rows: 64511
    })
})


In [68]:
data = data['train'].shard(num_shards=50, index=0).filter(lambda sample: sample['response'] != '' and sample['prompt'] != '').shuffle().map(generate_and_tokenize_prompt)


Filter:   0%|          | 0/5161 [00:00<?, ? examples/s]

Map:   0%|          | 0/5161 [00:00<?, ? examples/s]

In [ ]:
# finetune mất 2 tiếng rưỡi :V

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir="experiments",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.465000
2,1.393800
3,1.283100
4,1.261700
5,1.122300
6,1.592700
7,1.390300
8,0.834500
9,1.172700
10,0.944700


Step,Training Loss
1,1.465000
2,1.393800
3,1.283100
4,1.261700
5,1.122300
6,1.592700
7,1.390300
8,0.834500
9,1.172700
10,0.944700


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
%%time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Hỏi một câu về trường bằng tiếng anh thử tại data nó tiếng anh :V
prompt = """
<|im_start|>system
Bạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.
<|im_end|>
<|im_start|>user
Tell me about VNU-HCMUS university in VietNam
<|im_end|>
<|im_start|>assistant
""".strip()


encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Có nhiều yếu tố có thể ảnh hưởng đến chất lượng của một mô hình sau khi fine-tune. Dưới đây là một số yếu tố quan trọng:

1. **Số lượng dữ liệu huấn luyện**: Số lượng dữ liệu huấn luyện lớn thường dẫn đến một mô hình tốt hơn. Nếu có ít dữ liệu, mô hình có thể không học được đủ thông tin để đưa ra các dự đoán chính xác.

2. **Chất lượng dữ liệu huấn luyện**: Dữ liệu huấn luyện cần phải đủ đa dạng và phản ánh đúng các trường hợp mà mô hình có thể gặp phải trong quá trình sử dụng thực tế. Dữ liệu không chính xác, không cân đối hoặc không đại diện có thể dẫn đến mô hình kém chất lượng.

3. **Kiến trúc mô hình**: Lựa chọn kiến trúc mô hình phù hợp cho bài toán cũng rất quan trọng. Một số kiến trúc mô hình có thể hoạt động tốt hơn đối với một loại nhiệm vụ nhất định.

4. **Tham số huấn luyện**: Các tham số như kích thước batch, số lượng epoch, learning rate, và số lượng lớp ẩn có thể ảnh hưởng đến hiệu suất của mô hình sau khi fine-tune.

5. **Phương pháp tối ưu hóa**: Lựa chọn phương pháp tối ưu hóa (optimizer) và các siêu tham số của nó cũng có thể ảnh hưởng đến hiệu suất của mô hình.

6. **Kỹ thuật huấn luyện**: Sử dụng các kỹ thuật huấn luyện như dropout, regularization, hoặc kỹ thuật tăng cường dữ liệu cũng có thể cải thiện chất lượng của mô hình.

7. **Xử lý dữ liệu đầu vào**: Việc tiền xử lý dữ liệu có thể giúp loại bỏ nhiễu, chuẩn hóa dữ liệu và tăng tính tổng quát của mô hình.

8. **Kiểm định và tinh chỉnh**: Kiểm tra và điều chỉnh các siêu tham số của mô hình sẽ giúp tối ưu hóa hiệu suất của nó trên tập dữ liệu kiểm tra.

Một mô hình sau khi fine-tune sẽ được ảnh hưởng bởi nhiều yếu tố khác nhau và việc cân nhắc và tinh chỉnh các yếu tố này là rất quan trọng để đạt được hiệu suất tốt nhất.